### Demo of Critic Functionality on AmbigNQ task

In [1]:
from discussion_agents.cog.agent.critic import CriticAgent
from discussion_agents.cog.prompts.critic import CRITIC_INSTRUCTION_AMBIGNQ, CRITIC_CRITIQUE_INSTRUCTION_AMBIGNQ, AMBIGNQ_FEWSHOT_EXAMPLES_DIRECT, AMBIGNQ_FEWSHOT_EXAMPLES_COT, AMBIGNQ_FEWSHOT_EXAMPLES_CRITIC, AMBIGNQ_FEWSHOT_EXAMPLES_REACT

In [26]:
# API Keys were here, removed them for security reasons

In [3]:
# llm instantiation
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)

In [6]:
# google search wrapper instantiation
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
search_wrapper = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)

/home/jared/miniconda3/envs/discussion-agents/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.utilities.google_search.GoogleSearchAPIWrapper` was deprecated in langchain-community 0.0.33 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


In [8]:
# agent instantiation
agent = CriticAgent(llm=llm, search=search_wrapper)

Sample question pulled from the [AmbigNQ dataset](https://huggingface.co/datasets/ambig_qa?row=0)

In [25]:
sample_question = 'When did the simpsons first air on television?'

In [24]:
agent.generate(question=sample_question,
        examples=AMBIGNQ_FEWSHOT_EXAMPLES_REACT,
        prompt=CRITIC_INSTRUCTION_AMBIGNQ,
        critique_examples=AMBIGNQ_FEWSHOT_EXAMPLES_CRITIC,
        critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_AMBIGNQ
)

'The Simpsons first aired on television on December 17, 1989. So the answer is: December 17, 1989.'